## Lab 10 : R y Python


### @autor: Roberto Mendoza

In [175]:
# import libraries 

import pandas as pd 
import numpy as np
import re 
from tqdm import tqdm  # controlar el tiempo en un loop
import os
import statsmodels.api as sm  # linear regression utiliza todas las columnas de base de datos 
import statsmodels.formula.api as smf  # linear regression usa uan formula
from stargazer.stargazer import Stargazer
from sklearn import datasets, linear_model # models 
from sklearn.metrics import mean_squared_error, r2_score

# !pip install stargazer

user = os.getlogin()   # Username
os.chdir(f"C:/Users/{user}/Documents/GitHub/1ECO35_2022_2/Lab10") # Set directorio


In [55]:
# laod dataset

repdata = pd.read_stata(r"../data/dataverse_files/mss_repdata.dta",
                           convert_categoricals=False)

repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,muni,state,author,stconst,fh_civ,fh_pol,S,W,WoverS,soc
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0


In [56]:
# summmary statistics

repdata.describe()

# Tipo de variables

repdata.info()

repdata.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 742
Columns: 200 entries, ccode to soc
dtypes: datetime64[ns](1), float32(104), float64(78), int32(9), int8(5), object(3)
memory usage: 813.4+ KB


ccode                  float64
year            datetime64[ns]
country_name            object
country_code            object
GPCP                   float32
                     ...      
fh_pol                 float64
S                      float32
W                      float32
WoverS                 float32
soc                    float32
Length: 200, dtype: object

In [57]:
# Extraer year
# con month se puede extraer el mes 
# con day se puede extraer el día 

repdata['time_year'] = pd.DatetimeIndex(repdata['year']).year - 1978
repdata['time_year']

C:\Users\Roberto\AppData\Local\Temp\ipykernel_32060\3441985832.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  repdata['time_year'] = pd.DatetimeIndex(repdata['year']).year - 1978


0       3
1       4
2       5
3       6
4       7
       ..
738    17
739    18
740    19
741    20
742    21
Name: time_year, Length: 743, dtype: int64

In [58]:
dummys = pd.get_dummies(repdata["ccode"].astype(int), prefix = "ccode", dummy_na=False)
dummys.columns

# se convierte a entero repdata["ccode"].astype(int)

Index(['ccode_404', 'ccode_420', 'ccode_432', 'ccode_433', 'ccode_434',
       'ccode_435', 'ccode_436', 'ccode_437', 'ccode_438', 'ccode_439',
       'ccode_450', 'ccode_451', 'ccode_452', 'ccode_461', 'ccode_471',
       'ccode_475', 'ccode_481', 'ccode_482', 'ccode_483', 'ccode_484',
       'ccode_490', 'ccode_500', 'ccode_501', 'ccode_510', 'ccode_516',
       'ccode_517', 'ccode_520', 'ccode_522', 'ccode_530', 'ccode_540',
       'ccode_541', 'ccode_551', 'ccode_552', 'ccode_553', 'ccode_560',
       'ccode_565', 'ccode_570', 'ccode_571', 'ccode_572', 'ccode_580',
       'ccode_625'],
      dtype='object')

In [62]:
len(dummys.columns)

41

##### Reference get_dummies

https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [59]:
# Dummies a nivel país 

repdata = pd.concat([ repdata , dummys], axis = 1 )

In [60]:
repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,ccode_551,ccode_552,ccode_553,ccode_560,ccode_565,ccode_570,ccode_571,ccode_572,ccode_580,ccode_625
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0,0,0,0,0,0,0,0,0,0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0,0,0,0,0,0,0,0,0,0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0,0,0,0,0,0,0,0,0,0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0,0,0,0,0,0,0,0,0,0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,0,1,0,0,0,0,0,0,0,0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,0,1,0,0,0,0,0,0,0,0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,0,1,0,0,0,0,0,0,0,0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,0,1,0,0,0,0,0,0,0,0


In [67]:
# Creación del trend_country effects

i = 0

while i < 41:  # 41 por el tema de indexing pues en python la posición inicial es cero. 
    var = dummys.columns[i]+"_"+"time"  # creamos el nombre de cada variable
    repdata[var]  = repdata[dummys.columns[i]]*repdata["time_year"] # multiplicacón de variables

    i = i + 1

In [68]:
repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,ccode_551_time,ccode_552_time,ccode_553_time,ccode_560_time,ccode_565_time,ccode_570_time,ccode_571_time,ccode_572_time,ccode_580_time,ccode_625_time
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0,0,0,0,0,0,0,0,0,0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0,0,0,0,0,0,0,0,0,0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0,0,0,0,0,0,0,0,0,0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0,0,0,0,0,0,0,0,0,0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,0,17,0,0,0,0,0,0,0,0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,0,18,0,0,0,0,0,0,0,0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,0,19,0,0,0,0,0,0,0,0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,0,20,0,0,0,0,0,0,0,0


In [72]:
table1 = repdata.loc[:,["any_prio", "any_prio_on", "any_prio_off","war_prio", "war_prio_on", "war_prio_off",
                        "war_col", "war_inc", "war","GPCP", "GPCP_g", "GPCP_g_l","gdp_g", "gdp_g_l",
        "y_0", "polity2l", "polity2l_6", "ethfrac", "relfrac", "Oil", "lmtnest", "lpopl1", "tot_100_g"]]

table1 

,any_prio,any_prio_on,any_prio_off,war_prio,war_prio_on,war_prio_off,war_col,war_inc,war,GPCP,...,gdp_g_l,y_0,polity2l,polity2l_6,ethfrac,relfrac,Oil,lmtnest,lpopl1,tot_100_g
0,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,839.215759,...,0.019637,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.933400,NaN
1,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,969.864563,...,-0.037037,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.959697,NaN
2,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,936.421631,...,0.044615,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.985946,NaN
3,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,992.536255,...,-0.008837,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,9.012134,NaN
4,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,974.396118,...,0.026746,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,9.035987,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,493.448456,...,0.030296,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.285019,-0.006359
739,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,746.904236,...,-0.018779,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.306650,0.001000
740,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,780.051941,...,0.078490,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.327418,0.095405
741,1.0,1.0,NaN,1.0,1.0,NaN,0.0,0.0,0.0,638.486389,...,0.008137,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.347324,-0.006019


In [113]:
summary_table = table1.describe().loc[["mean","std","count"]].T

In [114]:
table1.columns

new_names = ["Civil conflict with ≥25 deaths: (PRIO/Uppsala)","Onset","Offset",
"Civil conflict with ≥1,000 deaths:PRIO/Uppsala","Onset","Offset","Collier and Hoeffler (2002)",  
"Doyle and Sambanis (2000)","Fearon and Laitin (2003)",
"Annual rainfall (mm), GPCP measure",
"Annual growth in rainfall, time t",
"Annual growth in rainfall, time t-1",
"Annual economic growth rate, time t",
"Annual economic growth rate, time t-1",
"Log(GDP per capita), 1979",
"Democracy level (Polity IV score, -10 to 10), time t-1",
"Democracy indicator (Polity IV score > 15),time t-1",
"Ethnolinguistic fractionalization (source:Atlas Marodov Mira)",
"Religious fractionalization (source: CIAFactbook)",
"Oil-exporting country (source: WDI)",
"Log(mountainous) (source: Fearon and Laitin 2003)",
"Log(national population), time t-1 (source: WDI)",
"Growth in terms of trade, time t (source:WDI)"]

# unión de listas bajo la estructura diccionario

dict( zip( table1.columns, new_names) )

{'any_prio': 'Civil conflict with ≥25 deaths: (PRIO/Uppsala)',
 'any_prio_on': 'Onset',
 'any_prio_off': 'Offset',
 'war_prio': 'Civil conflict with ≥1,000 deaths:PRIO/Uppsala',
 'war_prio_on': 'Onset',
 'war_prio_off': 'Offset',
 'war_col': 'Collier and Hoeffler (2002)',
 'war_inc': 'Doyle and Sambanis (2000)',
 'war': 'Fearon and Laitin (2003)',
 'GPCP': 'Annual rainfall (mm), GPCP measure',
 'GPCP_g': 'Annual growth in rainfall, time t',
 'GPCP_g_l': 'Annual growth in rainfall, time t-1',
 'gdp_g': 'Annual economic growth rate, time t',
 'gdp_g_l': 'Annual economic growth rate, time t-1',
 'y_0': 'Log(GDP per capita), 1979',
 'polity2l': 'Democracy level (Polity IV score, -10 to 10), time t-1',
 'polity2l_6': 'Democracy indicator (Polity IV score > 15),time t-1',
 'ethfrac': 'Ethnolinguistic fractionalization (source:Atlas Marodov Mira)',
 'relfrac': 'Religious fractionalization (source: CIAFactbook)',
 'Oil': 'Oil-exporting country (source: WDI)',
 'lmtnest': 'Log(mountainous) (sou

In [115]:
# Customize summary table 

index = dict( zip( table1.columns, new_names) )

columns = {
    "mean": "Mean",
    "std": "Standard Deviation",
    "count": "Observations",
}

formatters = {
    "mean": "{:.2f}".format,
    "std": "{:.2f}".format,
    "count": "{:.0f}".format,
}

# Rename rows (indexes) and columns
summary_table.rename(index=index, columns=columns, inplace=True)

# Export the DataFrame to LaTeX
summary_table.to_latex(
    "summary.tex",
    formatters={columns[c]: f for c, f in formatters.items()},
    index_names=False,
    column_format = "lccc",
    caption = "Descriptive Statistics"
)

C:\Users\Roberto\AppData\Local\Temp\ipykernel_32060\1751471283.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  summary_table.to_latex(


In [124]:

#Columns format

summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2).format(subset="Observations", precision=0)


# Export the DataFrame to LaTeX
# \ permite esccribir código extenso en lineas diferentes

summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2)\
.format(subset="Observations", precision=0)\
.to_latex(
    "summary2.tex",
caption="Descriptive Statistics",
    column_format = "lccc"
) 

In [123]:
summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2).format(subset="Observations", precision=0)


,Mean,Standard Deviation,Observations
Civil conflict with ≥25 deaths: (PRIO/Uppsala),0.267833,0.443128,743.0
Onset,0.068468,0.252776,555.0
Offset,0.143617,0.351638,188.0
"Civil conflict with ≥1,000 deaths:PRIO/Uppsala",0.166891,0.373129,743.0
Onset,0.036800,0.188421,625.0
Offset,0.144068,0.352656,118.0
Collier and Hoeffler (2002),0.169583,0.375518,743.0
Doyle and Sambanis (2000),0.220994,0.415203,724.0
Fearon and Laitin (2003),0.244953,0.430350,743.0
"Annual rainfall (mm), GPCP measure",1001.638367,501.701569,743.0


## First stage 

#### Linear models libraries

https://www.statsmodels.org/stable/index.html

In [164]:

# add constant

y = repdata['gdp_g']

X = sm.add_constant(repdata.loc[:,["GPCP_g", "GPCP_g_l"]])

# sm function

ols_model = sm.OLS(y, X).fit()

# fit permite correr la regresión

print(ols_model.summary())

# Robust standar error

ols_model_rb = sm.OLS(y, X).fit().get_robustcov_results(cov_type = "HC1")
print(ols_model_rb.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     8.912
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           0.000150
Time:                        17:23:45   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0061      0.003     -2.375      0.0

In [173]:
# Acceder a la información de la tabla

ols_model_rb.summary2()

ols_model_rb.summary2().tables[1]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
const,-0.006147,0.002617,-2.348553,0.019109,-0.011285,-0.001009
GPCP_g,0.055430,0.014180,3.908990,0.000101,0.027592,0.083268
GPCP_g_l,0.034058,0.011901,2.861785,0.004332,0.010694,0.057422


In [154]:
dir(sm.OLS(y, X))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_kwargs',
 '_data_attr',
 '_df_model',
 '_df_resid',
 '_fit_collinear',
 '_fit_ridge',
 '_fit_zeros',
 '_formula_max_endog',
 '_get_init_kwds',
 '_handle_data',
 '_init_keys',
 '_kwargs_allowed',
 '_setup_score_hess',
 '_sqrt_lasso',
 'data',
 'df_model',
 'df_resid',
 'endog',
 'endog_names',
 'exog',
 'exog_names',
 'fit',
 'fit_regularized',
 'from_formula',
 'get_distribution',
 'hessian',
 'hessian_factor',
 'information',
 'initialize',
 'k_constant',
 'loglike',
 'nobs',
 'predict',
 'rank',
 'score',
 'weights',
 'wendog',
 'wexog',
 'whiten']

In [157]:
# Recordad métodos y atributos 

dir(sm.OLS(y, X))

# predict para ello uso la función predict 

ols_model.predict(X)

# acceso a los parámetros

ols_model.params

# R2 y R2  ajustado

ols_model.rsquared
ols_model.rsquared_adj

0.020880268531097146

In [212]:
control_formula = "gdp_g"+ " ~ "+ "GPCP_g + " + "GPCP_g_l"

ols_model = smf.ols(control_formula, data=repdata).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     8.912
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           0.000150
Time:                        18:02:07   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0061      0.003     -2.375      0.0

In [153]:
# ´predict 

ols_model.predict(X)

0     -0.014226
1     -0.000230
2     -0.002756
3     -0.004000
4     -0.005119
         ...   
738   -0.013549
739    0.024408
740    0.013807
741   -0.014695
742   -0.007094
Length: 743, dtype: float64

### Using sklearn (modelos ML lineales)

https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py



In [201]:
ols_model_skl = linear_model.LinearRegression().fit( X, y )

ols_model_skl.coef_ # acceso a coeficientes 

ols_model_skl.predict(X) # predict en formato array 
ols_model_skl.score(X,y) # R cuadrado


0.023519415945896127

In [193]:
dir(ols_model_skl)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_preprocess_data',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_residues',
 '_set_intercept',
 '_validate_data',
 'coef_',
 'copy_X',
 'feature_names_in_',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'normalize',
 'positive',
 'predict',
 'rank_',
 'score',
 'set_params',
 'singular_']

In [186]:
mean_squared_error( y, ols_model.predict(X))**0.5 # root mean square error 

0.06981466645337808

## Modelo1 

- OLS, sin efectos fijos o country-time trend
- errores estandar robustas (Huber robust)
- Los residuos estan clusterizados (agrupados) a nivel país
-  Using cluster and robust standar error


In [206]:
control_formula = "gdp_g"+ " ~ "+ "GPCP_g + " + "GPCP_g_l"

ols_model1 = smf.ols(control_formula, data=repdata).fit().\
get_robustcov_results(cov_type = "cluster", groups= repdata['ccode'])


In [207]:
print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     6.672
Date:                Fri, 04 Nov 2022   Prob (F-statistic):            0.00316
Time:                        18:00:21   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:              cluster                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0061      0.002     -2.499      0.0

In [213]:
print(mean_squared_error( y, ols_model1.predict(X))**0.5
ols_model.rsquared

0.023519405273600857